Criado para para aprendizado de WebScrappping

A principio estava usando BeautifilSoap, que carregar a HTML da página (estática)
Porém como a página de de ASPX, com necessidade de  interação passei a fazer com biblioteca Selenium

5-mai-25 melhorou com contribuição do canal Comunidade DS: https://youtu.be/XLkxOBY965w
mas ainda tem bastante coisa pra acertar e muito a aprender.

6-mai-25 - Fechado o desenvolvimento, porém não existem os 43 itens informado na página, deve ser algum erro nsa apçlicação.
mas o desenvolvimento esta finalzido.


In [4]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
import gc
from selenium.webdriver.common.by import By
import pandas as pd

In [5]:
#Service é a classe usada para inicializar instancia do crome webdrive
service = Service()

#webdrive.ChromeOption é usado para definir a preferencia para browser cromoe
options = webdriver.ChromeOptions()

#inica-se a intancia do Chorme WebCriver com as definidas 'options' e 'saervice'
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.aeromercado.com.br/buscaprodA20.asp?tipo=acessorios"

driver.get(url)

#time.sleep(3)

In [6]:


#Carrega as 10 linhas em dados com nome do item a venda
def CarregaItens(qlin):
    dados = driver.find_elements(By.CLASS_NAME, "divAeromercado")[1:qlin+1:1]
    itens = []
    for row in dados:  
        text = row.text
        itens.append(text)
    return itens


In [7]:
#Carrega as 10 linhas em dados com preço do item a venda
def CarregaPrecos(qlin):
    dados = driver.find_elements(By.CLASS_NAME, "right4p")[0:qlin:1]
    precos = []
    for row in dados:  
        linhas = row.text.split("\n")  #Aqui tem 2 linhas, pegmaos a segunda"
        text = linhas[1]
        precos.append(text)
    return precos

In [8]:
# Fnção para concatenar as próximas páginas no DF
def concatDF(df, qtditem):

    itens = CarregaItens(qtditem)
    precos = CarregaPrecos(qtditem)
    novo_df = pd.DataFrame({
        'produto': itens,
        'preco': precos
    })

    dfresult = pd.concat([df, novo_df], ignore_index=True)
   
    return dfresult
    

In [ ]:
# Esta funcionando, mas o numero de itens informado, 43, na aparece na tela, a pagina 5 esta vazia, deve ter erro no pagina.
qtdAcessorios = driver.find_elements(By.CLASS_NAME, 'aencontradasop')
qtdAcessorios = int(qtdAcessorios[0].text.split()[0])  #pega a priemira linha e primeira palavra
qtdpag = qtdAcessorios // 10
sobra =  qtdAcessorios % 10
if sobra > 0: qtdpag = qtdpag + 1
ptdproc = 10
if qtdAcessorios <= 10: #Determina qtd da primeira pagina
    qtdproc = qtdAcessorios 

In [ ]:
qtdproc = 10
if qtdAcessorios < 10: #Determina qtd da primeira pagina
    qtdproc = qtdAcessorios 

#Carrega primeira pagina no DF
itens = CarregaItens(qtdproc)
precos = CarregaPrecos(qtdproc)
df = pd.DataFrame({
    'produto': itens,
    'preco': precos
})

In [13]:
qtdproc = 10
if qtdAcessorios < 10: #Determina qtd da primeira pagina
    qtdproc = qtdAcessorios 

#Carrega primeira pagina no DF
itens = CarregaItens(qtdproc)
precos = CarregaPrecos(qtdproc)
df = pd.DataFrame({
    'produto': itens,
    'preco': precos
})
# Se tiver mais de 10 itens tem mais páginas, então carrega as demais (a partir da segunda)    
if qtdpag > 1:
    for i in range(1, qtdpag):
    
        driver.find_elements(By.LINK_TEXT, "PRÓXIMA")[0].click()
        time.sleep(3)
        qtdproc = 10
        if sobra > 0 and qtdpag == i:
            qtdproc = sobra   
        df = concatDF(df, qtdproc)
        

#print(df)

In [ ]:
driver.find_element(By.LINK_TEXT, str(1)).click()

In [8]:
driver.find_elements(By.LINK_TEXT, "PRÓXIMA")[0].click()